In [4]:
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import ToTensor, Resize, Normalize, Compose
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from torch.optim import AdamW


# 데이터셋 클래스 정의
class KoreanTextDataset(Dataset):
    def __init__(self, image_dir, json_dir, transform=None, target_size=(384, 384)):
        self.image_dir = image_dir
        self.json_dir = json_dir
        self.transform = transform
        self.target_size = target_size  # 고정 크기 설정
        self.samples = self._load_data()

    def _load_data(self):
        data = []
        json_files = [f for f in os.listdir(self.json_dir) if f.endswith(".json")]
        for json_file in json_files:
            json_path = os.path.join(self.json_dir, json_file)
            image_path = os.path.join(self.image_dir, json_file.replace(".json", ".png"))
            if os.path.exists(image_path):
                # JSON 파일 읽을 때 UTF-8 BOM 문제 해결
                with open(json_path, "r", encoding="utf-8-sig") as f:
                    annotation = json.load(f)
                question_text = annotation["OCR_info"][0]["question_text"]
                bboxes = [
                    bbox["bbox"]
                    for bbox in annotation["OCR_info"][0]["question_bbox"]
                    if bbox["type"] == "line"
                ]
                data.append((image_path, bboxes, question_text))
        return data

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, bboxes, label = self.samples[idx]
        image = Image.open(image_path).convert("RGB")
        cropped_images = []
        for bbox in bboxes:
            x, y, w, h = bbox
            cropped_image = image.crop((x, y, x + w, y + h))  # 좌표로 크롭
            cropped_image = cropped_image.resize(self.target_size)  # 고정 크기로 리사이즈
            if self.transform:
                cropped_image = self.transform(cropped_image)
            cropped_images.append(cropped_image)
        return torch.stack(cropped_images), label


# 데이터셋 경로 설정
image_dir = "./x/test3"
json_dir = "./y/test3"

# 데이터셋 로드 및 분할
full_dataset = KoreanTextDataset(
    image_dir=image_dir,
    json_dir=json_dir,
    transform=Compose([
        ToTensor(),
        Normalize((0.5,), (0.5,))
    ]),
    target_size=(384, 384)  # 모델 입력 크기
)

# 데이터셋 분리 (80% Train, 20% Validation)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# 데이터 로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# 모델 초기화
processor = TrOCRProcessor.from_pretrained("team-lucid/trocr-small-korean")
model = VisionEncoderDecoderModel.from_pretrained("team-lucid/trocr-small-korean")

# 모델 설정
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

# 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 및 검증 루프
epochs = 10  # 에포크 수를 10으로 설정
for epoch in range(epochs):
    # 학습 단계
    model.train()
    total_train_loss = 0
    for cropped_images, ground_truth_text in train_dataloader:
        cropped_images = cropped_images.to(device)

        # 라벨을 토큰화하여 모델 입력으로 사용
        labels = processor.tokenizer(
            ground_truth_text, padding=True, truncation=True, return_tensors="pt"
        ).input_ids.to(device)

        outputs = model(pixel_values=cropped_images.squeeze(0), labels=labels)
        loss = outputs.loss
        total_train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_dataloader)

    # 검증 단계
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for cropped_images, ground_truth_text in val_dataloader:
            cropped_images = cropped_images.to(device)

            # 예측 수행: generate 메서드 사용
            predicted_ids = model.generate(pixel_values=cropped_images.squeeze(0))
            predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)

            # Ground Truth 출력 (JSON의 원본 텍스트 그대로)
            print("Ground Truth:", ground_truth_text[0])
            print("Predicted Texts:", predicted_text)

            # 검증 손실 계산
            labels = processor.tokenizer(
                ground_truth_text, padding=True, truncation=True, return_tensors="pt"
            ).input_ids.to(device)
            outputs = model(pixel_values=cropped_images.squeeze(0), labels=labels)
            loss = outputs.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_dataloader)

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Validation Loss: {avg_val_loss:.4f}")

# 학습된 모델 저장
model.save_pretrained("./trocr-korean-finetuned")
processor.save_pretrained("./trocr-korean-finetuned")

Config of the encoder: <class 'transformers.models.deit.modeling_deit.DeiTModel'> is overwritten by shared encoder config: DeiTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "model_type": "deit",
  "num_attention_heads": 6,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 384,
  "d_model": 256,
  "decoder_attention_heads": 8,
  "decod

Ground Truth: $296+403$에서 $296$과 $403$을 각각 가장 가까운 몇백으로 어림하여 계산해 보세요.
Predicted Texts: ['2013에서 $10$을 각각 가장 큰 몇백으로 하여 $10$으로 각각으로', ' $4$이 $1$이 $1$이 $1$이 $1$']
Ground Truth: $327+251$에서 $327$과 $251$을 각각 가장 가까운 몇백 몇십으로 어림하여 계산해 보세요.
Predicted Texts: ['$2$1$과 $251$을 각각 계산해 보세요. $5$', ' $4$ $5$ $10$ $10$ $10$ $10$이']
Ground Truth: 다음이 나타내는 수보다 $284$만큼 더 큰 수를 구해 보세요. $100$이 $3$ 개, $10$이 $5$ 개, $1$이 $8$ 개인 수
Predicted Texts: ['다음이 각각으로 구해 보세요. $6$이 $6$이 $6$', ' $1$이 $1$이 $1$이는 $1$이는']
Ground Truth: 다음에서 짝수를 찾아 짝수들의 합을 구해 보세요. $275$ $176$ $293$ $387$ $618$
Predicted Texts: ['다음에서 짝수를 구해 보세요.', ' $1$이 $1$이 $1$이는 $1$이는']
Ground Truth: 다음에서 홀수를 찾아 홀수들의 합을 구해 보세요. $127$ $426$ $745$ $252$ $328$
Predicted Texts: ['다음으로 보세요. $1$ $1$ $1$ $1$ $1', ' $1$이 $1$이 $1$이는 $1$이는']
Ground Truth: $523+316$에서 $523$과 $316$을 각각 가장 가까운 몇백 몇십으로 어림하여 계산해 보세요.
Predicted Texts: ['$2$에서 $2$이 $6$을 각각 계산해 보세요. $', ' $4$ $5$ $10$ $10$ $10$ $10$이']
Ground Truth: 재영이는 집에서 출발하여 서점에 들렀다가 학교에 가려고 합니다. 재영이가 걸어야 하는 거리는 몇 $m$인지 구해

[]